# DEPLOY

In [30]:
import pandas as pd
print('pandas', pd.__version__)

import numpy as np

import pickle
print('pickle', pickle.format_version)

import math

pandas 1.3.5
pickle 4.0


# LIVELLO

## DATASET LOADING

In [31]:
df = pd.read_csv('###', sep=';')
df.head()
# Noi abbiamo bisogno di una serie che pero' ci permetta anche di avere i lag indietro degli attributi
# Fatto questo li lagghiamo
# Poi la previsione e' come nel test

,Data,Pioggia_Zona_1,Pioggia_Zona_2,Pioggia_Zona_3,Pioggia_Zona_4,Pioggia_Zona_5,Temperatura_Zona_5,Livello_Acqua,Portata_Uscita
0,07/07/2020,0.0,0.0,0.0,0.0,0.0,22.95,29.82,0.6
1,08/07/2020,0.0,0.0,0.0,0.0,0.1,21.50,29.86,0.5
2,09/07/2020,0.0,0.0,0.0,0.1,0.0,21.95,29.86,0.6
3,10/07/2020,0.0,0.0,0.0,0.0,0.0,23.00,29.87,0.6
4,11/07/2020,0.0,0.0,0.0,0.0,0.0,22.85,29.88,0.6


## DATASET PREPARATION

In [32]:
# Creiamo attributo Pioggia_Totale dato la somma di tutta la pioggia caduta nelle varie zone
colonne_sum = ['Pioggia_Zona_1','Pioggia_Zona_2','Pioggia_Zona_3','Pioggia_Zona_4','Pioggia_Zona_5']
df = df.assign(Pioggia_Totale=df.loc[:,colonne_sum].sum(axis=1))
df.head()

,Data,Pioggia_Zona_1,Pioggia_Zona_2,Pioggia_Zona_3,Pioggia_Zona_4,Pioggia_Zona_5,Temperatura_Zona_5,Livello_Acqua,Portata_Uscita,Pioggia_Totale
0,07/07/2020,0.0,0.0,0.0,0.0,0.0,22.95,29.82,0.6,0.0
1,08/07/2020,0.0,0.0,0.0,0.0,0.1,21.50,29.86,0.5,0.1
2,09/07/2020,0.0,0.0,0.0,0.1,0.0,21.95,29.86,0.6,0.1
3,10/07/2020,0.0,0.0,0.0,0.0,0.0,23.00,29.87,0.6,0.0
4,11/07/2020,0.0,0.0,0.0,0.0,0.0,22.85,29.88,0.6,0.0


In [33]:
# Scaler loading
with open('###', 'rb') as file:
    standard_scaler_livello = pickle.load(file)

In [34]:
numerical_features = ['Pioggia_Totale', 'Temperatura_Zona_5']

df_norm = pd.DataFrame(
    standard_scaler_livello.fit_transform(df[numerical_features]),
    columns=numerical_features,
    index=df.index
    )

df_norm['Data'] = df['Data']
df_norm['Livello_Acqua'] = df['Livello_Acqua']

df_norm.head()
X = df_norm.copy()

In [35]:
X = X.loc[0:7]
X['Livello_Acqua'] = X['Livello_Acqua'].shift(-7)
X

,Pioggia_Totale,Temperatura_Zona_5,Data,Livello_Acqua
0,-0.648886,0.363907,07/07/2020,29.83
1,0.811107,-2.135561,08/07/2020,NaN
2,0.811107,-1.359864,09/07/2020,NaN
3,-0.648886,0.450096,10/07/2020,NaN
4,-0.648886,0.191530,11/07/2020,NaN
5,-0.648886,0.019153,12/07/2020,NaN
6,2.271100,0.794850,13/07/2020,NaN
7,-0.648886,1.139604,14/07/2020,NaN


In [36]:
# Si deve partire nel primo range da dove finiscono i dati 
num_sinusoidi = 2
omega = np.outer(range(1, 2), range(1,num_sinusoidi+1)) * 2 * math.pi / (365.25)
cc = np.cos(omega)
ss = np.sin(omega)

df_livello_passato = pd.DataFrame({})

for i in range(num_sinusoidi):
    df_livello_passato['cc_'+str(i+1)] = cc[:, i]
    df_livello_passato['ss_'+str(i+1)] = ss[:, i]

df_livello_passato['Pioggia_Totale'] = X.loc[:1, 'Pioggia_Totale']
df_livello_passato['Temperatura_Zona_5'] = X.loc[:1, 'Temperatura_Zona_5']

df_livello_passato

,cc_1,ss_1,cc_2,ss_2,Pioggia_Totale,Temperatura_Zona_5
0,0.999852,0.017202,0.999408,0.034398,-0.648886,0.363907


## MODEL LOADING

In [37]:
with open('../Models/Arima_Livello_Acqua.pkl', 'rb') as file:
    model_livello_arima = pickle.load(file)
model_livello_arima

## PREDICTION

In [38]:
new_obs = np.log(X.loc[0]['Livello_Acqua'])

In [39]:
new_obs = [new_obs]

In [40]:
# Come nel test, ovviamento con gli exog da sistemare
fit_newobs = model_livello_arima.apply(new_obs, exog = df_livello_passato)
fit_newobs

In [41]:
# FUTURO
num_sinusoidi = 2
omega = np.outer(range(1, 8), range(1,num_sinusoidi+1)) * 2 * math.pi / (365.25)
cc = np.cos(omega)
ss = np.sin(omega)

df_livello_futuro = pd.DataFrame({})

for i in range(num_sinusoidi):
    df_livello_futuro['cc_'+str(i+1)] = cc[:, i]
    df_livello_futuro['ss_'+str(i+1)] = ss[:, i]


df_livello_futuro['Pioggia_Totale'] = X.loc[:7, 'Pioggia_Totale']
df_livello_futuro['Temperatura_Zona_5'] = X.loc[:7, 'Temperatura_Zona_5']


df_livello_futuro

,cc_1,ss_1,cc_2,ss_2,Pioggia_Totale,Temperatura_Zona_5
0,0.999852,0.017202,0.999408,0.034398,-0.648886,0.363907
1,0.999408,0.034398,0.997634,0.068755,0.811107,-2.135561
2,0.998669,0.051584,0.994678,0.103031,0.811107,-1.359864
3,0.997634,0.068755,0.990545,0.137185,-0.648886,0.450096
4,0.996303,0.085906,0.985240,0.171177,-0.648886,0.191530
5,0.994678,0.103031,0.978769,0.204966,-0.648886,0.019153
6,0.992759,0.120126,0.971139,0.238513,2.271100,0.794850


In [42]:
Y_pred = np.exp(fit_newobs.forecast(steps = 7, exog = df_livello_futuro))

In [43]:
pred_livello = pd.DataFrame({})
pred_livello['Livello_Acqua'] = [Y_pred.loc[7]]
pred_livello

,Livello_Acqua
0,32.440931


## PREDICTION PERSISTENCE

In [44]:
pred_livello.to_csv('###', index=False)

# PORTATA

## DATASET LOADING

In [45]:
df = pd.read_csv('###', sep=';')
df
# Noi abbiamo bisogno di una serie che pero' ci permetta anche di avere i lag indietro degli attributi
# Fatto questo li lagghiamo
# Poi la previsione e' come nel test

,Data,Pioggia_Zona_1,Pioggia_Zona_2,Pioggia_Zona_3,Pioggia_Zona_4,Pioggia_Zona_5,Temperatura_Zona_5,Livello_Acqua,Portata_Uscita
0,07/07/2020,0.0,0.0,0.0,0.0,0.0,22.95,29.82,0.6
1,08/07/2020,0.0,0.0,0.0,0.0,0.1,21.50,29.86,0.5
2,09/07/2020,0.0,0.0,0.0,0.1,0.0,21.95,29.86,0.6
3,10/07/2020,0.0,0.0,0.0,0.0,0.0,23.00,29.87,0.6
4,11/07/2020,0.0,0.0,0.0,0.0,0.0,22.85,29.88,0.6
5,12/07/2020,0.0,0.0,0.0,0.0,0.0,22.75,29.86,0.6
6,13/07/2020,0.0,0.2,0.0,0.0,0.0,23.20,29.86,0.6
7,14/07/2020,0.0,0.0,0.0,0.0,0.0,23.40,29.83,0.6
8,15/07/2020,0.0,0.0,0.0,0.0,0.0,23.05,29.82,0.5


## DATASET PREPARATION

In [46]:
# Creiamo attributo Pioggia_Totale dato la somma di tutta la pioggia caduta nelle varie zone
colonne_sum = ['Pioggia_Zona_1','Pioggia_Zona_2','Pioggia_Zona_3','Pioggia_Zona_4','Pioggia_Zona_5']
df = df.assign(Pioggia_Totale=df.loc[:,colonne_sum].sum(axis=1))
df.head()

,Data,Pioggia_Zona_1,Pioggia_Zona_2,Pioggia_Zona_3,Pioggia_Zona_4,Pioggia_Zona_5,Temperatura_Zona_5,Livello_Acqua,Portata_Uscita,Pioggia_Totale
0,07/07/2020,0.0,0.0,0.0,0.0,0.0,22.95,29.82,0.6,0.0
1,08/07/2020,0.0,0.0,0.0,0.0,0.1,21.50,29.86,0.5,0.1
2,09/07/2020,0.0,0.0,0.0,0.1,0.0,21.95,29.86,0.6,0.1
3,10/07/2020,0.0,0.0,0.0,0.0,0.0,23.00,29.87,0.6,0.0
4,11/07/2020,0.0,0.0,0.0,0.0,0.0,22.85,29.88,0.6,0.0


In [47]:
# Scaler loading
with open('###', 'rb') as file:
    standard_scaler_portata = pickle.load(file)

In [48]:
numerical_features = ['Pioggia_Totale', 'Temperatura_Zona_5', 'Livello_Acqua']

df_norm = pd.DataFrame(
    standard_scaler_livello.fit_transform(df[numerical_features]),
    columns=numerical_features,
    index=df.index
    )

df_norm['Data'] = df['Data']
df_norm['Portata_Uscita'] = df['Portata_Uscita']

df_norm.head()
X = df_norm.copy()

In [49]:
X = X.loc[0:7]
X['Portata_Uscita'] = X['Portata_Uscita'].shift(-7)
X

,Pioggia_Totale,Temperatura_Zona_5,Livello_Acqua,Data,Portata_Uscita
0,-0.648886,0.363907,-1.496663,07/07/2020,0.6
1,0.811107,-2.135561,0.427618,08/07/2020,NaN
2,0.811107,-1.359864,0.427618,09/07/2020,NaN
3,-0.648886,0.450096,0.908688,10/07/2020,NaN
4,-0.648886,0.191530,1.389758,11/07/2020,NaN
5,-0.648886,0.019153,0.427618,12/07/2020,NaN
6,2.271100,0.794850,0.427618,13/07/2020,NaN
7,-0.648886,1.139604,-1.015593,14/07/2020,NaN


In [50]:
num_sinusoidi = 2
omega = np.outer(range(1, 2), range(1,num_sinusoidi+1)) * 2 * math.pi / (365.25)
cc = np.cos(omega)
ss = np.sin(omega)

df_portata_passato = pd.DataFrame({})

for i in range(num_sinusoidi):
    df_portata_passato['cc_'+str(i+1)] = cc[:, i]
    df_portata_passato['ss_'+str(i+1)] = ss[:, i]

df_portata_passato['Pioggia_Totale'] = X.loc[:1, 'Pioggia_Totale']
df_portata_passato['Temperatura_Zona_5'] = X.loc[:1, 'Temperatura_Zona_5']
df_portata_passato['Livello_Acqua'] = X.loc[:1, 'Livello_Acqua']
df_portata_passato['Livello_Acqua_square'] = df_portata_passato['Livello_Acqua']**2

t = np.array(range(1,len(df_portata_passato)+1))

df_portata_passato['Time'] = np.sin(np.log(t+1))


df_portata_passato

,cc_1,ss_1,cc_2,ss_2,Pioggia_Totale,Temperatura_Zona_5,Livello_Acqua,Livello_Acqua_square,Time
0,0.999852,0.017202,0.999408,0.034398,-0.648886,0.363907,-1.496663,2.24,0.638961


## MODEL LOADING

In [51]:
with open('###', 'rb') as file:
    model_portata_arima = pickle.load(file)
model_portata_arima

## PREDICTION

In [52]:
new_obs = np.log(X.loc[0]['Portata_Uscita'])

In [53]:
new_obs = [new_obs]

In [54]:
# Come nel test, ovviamento con gli exog da sistemare
fit_newobs = model_portata_arima.apply(new_obs, exog = df_portata_passato)
fit_newobs

In [55]:
num_sinusoidi = 2
omega = np.outer(range(1, 8), range(1,num_sinusoidi+1)) * 2 * math.pi / (365.25)
cc = np.cos(omega)
ss = np.sin(omega)

df_portata_futuro = pd.DataFrame({})

for i in range(num_sinusoidi):
    df_portata_futuro['cc_'+str(i+1)] = cc[:, i]
    df_portata_futuro['ss_'+str(i+1)] = ss[:, i]

df_portata_futuro['Pioggia_Totale'] = X.loc[:7, 'Pioggia_Totale']
df_portata_futuro['Temperatura_Zona_5'] = X.loc[:7, 'Temperatura_Zona_5']
df_portata_futuro['Livello_Acqua'] = X.loc[:7, 'Livello_Acqua']
df_portata_futuro['Livello_Acqua_square'] = df_portata_futuro['Livello_Acqua']**2

t = np.array(range(1,len(df_portata_futuro)+1))

df_portata_futuro['Time'] = np.sin(np.log(t+1))


df_portata_futuro

,cc_1,ss_1,cc_2,ss_2,Pioggia_Totale,Temperatura_Zona_5,Livello_Acqua,Livello_Acqua_square,Time
0,0.999852,0.017202,0.999408,0.034398,-0.648886,0.363907,-1.496663,2.240000,0.638961
1,0.999408,0.034398,0.997634,0.068755,0.811107,-2.135561,0.427618,0.182857,0.890577
2,0.998669,0.051584,0.994678,0.103031,0.811107,-1.359864,0.427618,0.182857,0.983028
3,0.997634,0.068755,0.990545,0.137185,-0.648886,0.450096,0.908688,0.825714,0.999254
4,0.996303,0.085906,0.985240,0.171177,-0.648886,0.191530,1.389758,1.931429,0.975687
5,0.994678,0.103031,0.978769,0.204966,-0.648886,0.019153,0.427618,0.182857,0.930466
6,0.992759,0.120126,0.971139,0.238513,2.271100,0.794850,0.427618,0.182857,0.873405


In [56]:
Y_pred = np.exp(fit_newobs.forecast(steps = 7, exog = df_portata_futuro))

In [57]:
pred_portata = pd.DataFrame({})
pred_portata['Portata_Uscita'] = [Y_pred.loc[7]]
pred_portata

,Portata_Uscita
0,0.035478


## PREDICTION PERCISTANCE

In [58]:
pred_portata.to_csv('###', index=False)